In [2]:
from robotsearch.robots import rct_robot
import numpy as np
import os
import pandas as pd

## Prepping CoronaWhy dataset

In [5]:
coy_df = pd.read_csv('/media/axhue/WD/Data/Coronawhy/Annotationv2.csv')

def clean_labels(labels):
    labels.rename(columns=labels.iloc[0,5:-1].to_dict(),inplace=True)
    labels.drop(0,inplace=True)
    designs = labels.columns[5:-1]

    labels.dropna(how='all',subset=designs,inplace=True)

    labels.fillna(0,inplace=True)

    labels.drop(labels.tail(1).index,inplace=True)
    labels.drop(columns=['Assignee','General Notes'],inplace=True)
    
clean_labels(coy_df)

In [33]:
#from ..datasets.specter_embeddings import annotations_with_specter_embeddings

#relative importing is a pain
def annotations_with_specter_embeddings(annotations_dataset, specter_embeddings_filepath):
    enriched_dataset = annotations_dataset.copy()
    specter_embeddings = pd.read_csv(
        specter_embeddings_filepath, header=None,
        names=['cord_uid'] + [f"{i}" for i in range(768)])
    enriched_dataset = enriched_dataset.merge(specter_embeddings, left_on="cord_uid", right_on="cord_uid")

    return enriched_dataset

In [35]:
coy_df = annotations_with_specter_embeddings(coy_df,'/media/axhue/WD/Data/Coronawhy/cord_19_embeddings_4_24.csv')

##  Prep Kaggle Dataset

In [6]:
kag_df = pd.read_csv('/media/axhue/WD/Data/Coronawhy/design.csv')

In [7]:
metadf = pd.read_csv('/media/axhue/WD/Data/Coronawhy/dataV8/metadata.csv')

In [10]:
id_map = dict(zip(metadf.sha,metadf.cord_uid))

In [12]:
cuid_df = kag_df.id.map(id_map,na_action='ignore')

In [13]:
no_corduid = kag_df[cuid_df.isna()].copy()

In [14]:
kag_df.id = cuid_df

In [15]:
kag_df.drop(index=no_corduid.index,inplace=True)

In [16]:
kag_df.rename(columns={'id':'cord_uid'},inplace=True)

## Eval RCT_Robot

In [17]:
rct_clf = rct_robot.RCTRobot()

In [18]:
rcts = kag_df[kag_df.label == 2]

In [19]:
len(coy_df[coy_df.RCT == '1'].loc[:,['cord_uid','RCT']])

15

In [20]:
merged_rcts = coy_df[coy_df.RCT == '1'].loc[:,['cord_uid','RCT']].merge(rcts,on='cord_uid',how='outer')

In [21]:
merged_rcts

,cord_uid,RCT,label
0,fakmw3cm,1,NaN
1,bkyf12lx,1,NaN
2,4qwp3um1,1,NaN
3,mpoqcfja,1,NaN
4,jgaxbhqk,1,NaN
...,...,...,...
65,e6aszi2a,NaN,2.0
66,j35tdxgt,NaN,2.0
67,ywlig1wl,NaN,2.0
68,p8vhi6d9,NaN,2.0


In [22]:
texts = metadf[metadf.cord_uid.isin(merged_rcts.cord_uid.values)]

In [23]:
preds = []
for i,t in texts.iterrows():
    pred = rct_clf.predict({"title": t['title'], 
                        "abstract": t['abstract'], 
                        "use_ptyp": False}, filter_type="balanced", filter_class="svm_cnn")
    preds.append(pred[0])

In [25]:
pd_df = pd.DataFrame(preds)

In [26]:
pd_df.head()

,model,score,threshold_type,threshold_value,is_rct,ptyp_rct,preds
0,svm_cnn,4.109292,balanced,1.649861,True,-1,"{'ptyp': 0.0, 'svm': 4.9757299079700035, 'svm_..."
1,svm_cnn,3.690826,balanced,1.649861,True,-1,"{'ptyp': 0.0, 'svm': 3.5663590894016064, 'svm_..."
2,svm_cnn,3.900710,balanced,1.649861,True,-1,"{'ptyp': 0.0, 'svm': 4.679049401396265, 'svm_p..."
3,svm_cnn,-0.546574,balanced,1.649861,False,-1,"{'ptyp': 0.0, 'svm': -0.2945804950977056, 'svm..."
4,svm_cnn,4.239765,balanced,1.649861,True,-1,"{'ptyp': 0.0, 'svm': 5.444579122147202, 'svm_p..."


In [151]:
pd_df.is_rct

0      True
1      True
2      True
3     False
4      True
      ...  
65     True
66     True
67     True
68     True
69    False
Name: is_rct, Length: 70, dtype: bool

In [150]:
from sklearn import metrics

In [152]:
metrics.f1_score([1 for i in range(len(merged_rcts))],
                 pd_df.is_rct.values)

0.8976377952755905

# Continue to ensemble with Catboost

## Training Catboost

In [156]:
total_data = kag_df[kag_df.label != 2].copy()

In [157]:
annotations_with_specter_embeddings(total_data,
                                    '/media/axhue/WD/Data/Coronawhy/cord_19_embeddings_4_24.csv')

,label,cord_uid,0,1,2,3,4,5,6,7,...,758,759,760,761,762,763,764,765,766,767
0,0,5k0ktboh,-1.385126,-4.899480,2.274657,1.361309,0.520716,-4.002396,1.116006,-1.436913,...,3.666709,0.858302,-3.915951,1.246150,-3.415432,0.671428,2.821265,-7.876527,0.134412,-1.122393
1,0,s7k8ldrj,1.427571,-3.049535,0.018854,-0.265127,1.696956,-0.383842,0.864027,-3.604572,...,-1.074684,-0.081458,-0.030828,1.118028,-0.360621,-0.728614,2.287663,-3.295221,-3.324802,-1.050134
2,0,5gkp9tlt,0.340833,-6.170926,4.844032,1.417662,1.216201,-0.783369,0.099707,-0.502377,...,-0.574510,3.270269,0.057638,-2.653418,-4.331059,-2.442574,1.261401,4.047961,0.307524,1.812008
3,0,e55vmgjr,-4.135505,-3.238813,0.930671,5.027368,1.397133,-2.105466,-0.686213,2.159149,...,-2.215385,2.250563,1.973086,1.840072,-3.441311,-5.703286,4.132703,1.539255,-0.133712,0.040155
4,0,270msv5l,0.012790,-3.482888,-0.669372,5.571725,1.196127,-2.706384,0.310297,-1.874102,...,1.312426,-0.537052,-2.960310,4.175011,0.551078,-3.045691,2.928529,-11.142795,-0.195732,2.133401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,9,tvxaxfqv,-6.925681,-6.446629,0.023540,3.581105,-0.378659,-3.753767,0.150186,1.829936,...,-3.656626,0.604885,-1.719747,-2.595123,-0.370621,-1.979728,2.848536,2.035826,-1.112365,1.162631
1063,9,qs7dsxn5,-3.991997,-6.602211,-0.630394,4.518262,0.745782,-4.451550,2.193783,0.588708,...,-2.370848,1.587442,1.987333,-0.684319,-1.205862,-3.276368,4.174228,0.952112,1.610153,1.951829
1064,9,qyf59ghf,-0.378375,-5.156946,4.397181,4.587927,1.801470,-4.129900,2.084496,1.033599,...,0.927953,0.687402,1.026297,0.716124,-1.791637,-6.037010,0.689584,-2.088232,1.206452,-0.061950
1065,9,kifqgskc,-5.008198,-2.767369,-2.950863,3.873948,-1.890941,-2.062921,-0.659847,2.304827,...,-2.632959,4.482713,1.502051,2.954271,1.343477,-2.854861,4.832151,3.176678,1.477337,1.978232


In [44]:
from pathlib import Path
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from catboost import CatBoostClassifier, Pool

# https://github.com/catboost/tutorials/blob/2c16945f850503bfaa631176e87588bc5ce0ca1c/text_features/text_features_in_catboost.ipynb


def train_validate_catboost_model(train_data, test_data, train_features, target_feature, text_features, params, verbose=True):
    X_train = train_data[train_features]
    y_train = train_data[target_feature]
    X_test = test_data[train_features]
    y_test = test_data[target_feature]

    train_pool = Pool(
        X_train,
        y_train,
        feature_names=list(train_features),
        text_features=text_features)
    test_pool = Pool(
        X_test,
        y_test,
        feature_names=list(train_features),
        text_features=text_features)

    catboost_default_params = {
        'iterations': 1000,
        'learning_rate': 0.03,
        'eval_metric': 'Accuracy',
        'task_type': 'GPU'
    }
    params = {**catboost_default_params, **params}
    model = CatBoostClassifier(**params)
    model.fit(train_pool, eval_set=test_pool, verbose=verbose)
    prediction = model.predict(test_pool)
    acc = accuracy_score(y_test, prediction)
    f1 = f1_score(y_test, prediction, average=None)
    f1_macro = f1_score(y_test, prediction, average="macro")
    logger.info(f"accuracy: {acc}")
    logger.info(f"f1: {f1}")
    logger.info(f"f1_macro: {f1_macro}")
    return model, acc, f1, f1_macro, params

In [158]:
def single_label_multiclass_annotated_study_design(annotations_filepath, metadata_filepath):
    """
    match david's annotated dataset (colums id, label) with the article metadata
    file from kaggle cord-19 https://www.kaggle.com/davidmezzetti/cord19-study-design
    :param annotations_filepath:
    :param metadata_filepath:
    :return: pd.DataFrame of annotations, with columns ["sha", "cord_uid",  "title", "abstract", "label", "label_str"]
    """

    label_number_to_study_name_mapping = {
        1: "Meta analysis", #systematic review and meta-analysis
        2: "Randomized control trial",
        3: "Non-randomized trial",
        4: "Prospective cohort", #prospective observational study
        5: "Time-series analysis",
        6: "Retrospective cohort", #retrospective observational study
        7: "Cross-sectional", # cross sectional study
        8: "Case control",
        9: "Case study", #case series
        10: "Simulation", #simulation
        0: "Other"
    }
    annotations = pd.read_csv(annotations_filepath)
    cord19_metadata = pd.read_csv(metadata_filepath)

    annotations = annotations.rename(columns={"id": "sha"}).dropna()
    annotations = annotations.merge(cord19_metadata, left_on="sha", right_on="sha")[
        ["sha", "cord_uid", "label", "title", "abstract"]]
    annotations = annotations.dropna(subset=['abstract', 'title']).drop_duplicates(
        subset=['abstract', 'title'])
    annotations['label_string'] = annotations["label"].apply(label_number_to_study_name_mapping.get)

    return annotations

In [181]:
def train_validate_catboost_model(train_data, test_data, train_features, target_feature, text_features, params, verbose=True):
    X_train = train_data[train_features]
    y_train = train_data[target_feature]
    X_test = test_data[train_features]
    y_test = test_data[target_feature]

    train_pool = Pool(
        X_train,
        y_train,
        feature_names=list(train_features),
        text_features=text_features)
    test_pool = Pool(
        X_test,
        y_test,
        feature_names=list(train_features),
        text_features=text_features)

    catboost_default_params = {
        'iterations': 1000,
        'learning_rate': 0.03,
        'eval_metric': 'Accuracy',
        'task_type': 'GPU'
    }
    params = {**catboost_default_params, **params}
    model = CatBoostClassifier(**params)
    model.fit(train_pool, eval_set=test_pool, verbose=verbose)
    prediction = model.predict(test_pool)
    acc = accuracy_score(y_test, prediction)
    f1 = f1_score(y_test, prediction, average=None)
    f1_macro = f1_score(y_test, prediction, average="macro")
    print(f"accuracy: {acc}")
    print(f"f1: {f1}")
    print(f"f1_macro: {f1_macro}")
    return model, acc, f1, f1_macro, params

In [171]:
total_data = total_data.merge(metadf.loc[:,['cord_uid','title','abstract']])

In [220]:
total_data.label.value_counts()

0    470
9    168
8    156
7    105
1     70
6     44
4     29
5     22
3      3
Name: label, dtype: int64

In [200]:
filt_dat = total_data.dropna()

In [201]:
len(filt_dat)

822

In [202]:
mapp = {0:0,1:1}
for i in range(3,10):
    mapp[i] = i-1
print(mapp)

{0: 0, 1: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8}


In [203]:
filt_dat.head()

,label,cord_uid,title,abstract
2,0,5gkp9tlt,Characteristic ECG Manifestations in Patients ...,Abstract Cardiac involvement has been reported...
3,0,e55vmgjr,COVID-19 admissions calculators: General popul...,Abstract The world is in the grip of pandemic ...
6,0,kewnvb7p,COVID-19 the showdown for mass casualty prepar...,"Since December 2019, the world is potentially ..."
11,0,52nw9gxq,An artificial intelligence-based first-line de...,ABSTRACTTo combat the pandemic of the coronavi...
12,0,ld0vo1rl,COVID-19 coronavirus vaccine design using reve...,To ultimately combat the emerging COVID-19 pan...


In [204]:
filt_dat.label = filt_dat.label.map(mapp,na_action='ignore').values

/home/axhue/miniconda2/envs/rbtsearch/lib/python3.6/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [214]:
filt_dat.label.unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [179]:
from sklearn.metrics import accuracy_score, f1_score
from catboost import CatBoostClassifier, Pool

In [215]:
results = []
params = {}
filt_dat = filt_dat.sample(frac=1.0)
test_set_len = int(len(filt_dat)/5)

for i in range(5):
    test = filt_dat.iloc[i * test_set_len : (i+1)* test_set_len]
    train = filt_dat[~filt_dat.index.isin(test.index)]
    model, acc, f1, f1_macro, params = train_validate_catboost_model(
        train, test,
        ['abstract', 'title'],
        'label',
        text_features=['abstract', 'title'], params={})
    results.append({
        "i": i,
        "f1": f1,
        "acc": acc,
        'f1_macro': f1_macro,
    })

0:	learn: 0.6322188	test: 0.6158537	best: 0.6158537 (0)	total: 13.4ms	remaining: 13.4s
1:	learn: 0.6398176	test: 0.6585366	best: 0.6585366 (1)	total: 26.9ms	remaining: 13.4s
2:	learn: 0.6352584	test: 0.6402439	best: 0.6585366 (1)	total: 39.1ms	remaining: 13s
3:	learn: 0.6428571	test: 0.6524390	best: 0.6585366 (1)	total: 53ms	remaining: 13.2s
4:	learn: 0.6565350	test: 0.6463415	best: 0.6585366 (1)	total: 68.8ms	remaining: 13.7s
5:	learn: 0.6595745	test: 0.6585366	best: 0.6585366 (1)	total: 80.3ms	remaining: 13.3s
6:	learn: 0.6626140	test: 0.6646341	best: 0.6646341 (6)	total: 91.6ms	remaining: 13s
7:	learn: 0.6580547	test: 0.6585366	best: 0.6646341 (6)	total: 104ms	remaining: 12.8s
8:	learn: 0.6610942	test: 0.6646341	best: 0.6646341 (6)	total: 114ms	remaining: 12.6s
9:	learn: 0.6595745	test: 0.6707317	best: 0.6707317 (9)	total: 127ms	remaining: 12.6s
10:	learn: 0.6550152	test: 0.6524390	best: 0.6707317 (9)	total: 139ms	remaining: 12.5s
11:	learn: 0.6595745	test: 0.6463415	best: 0.6707317

95:	learn: 0.7598784	test: 0.7073171	best: 0.7134146 (42)	total: 1.02s	remaining: 9.57s
96:	learn: 0.7598784	test: 0.7073171	best: 0.7134146 (42)	total: 1.02s	remaining: 9.55s
97:	learn: 0.7629179	test: 0.7073171	best: 0.7134146 (42)	total: 1.04s	remaining: 9.58s
98:	learn: 0.7598784	test: 0.7073171	best: 0.7134146 (42)	total: 1.05s	remaining: 9.56s
99:	learn: 0.7613982	test: 0.7073171	best: 0.7134146 (42)	total: 1.06s	remaining: 9.54s
100:	learn: 0.7629179	test: 0.7073171	best: 0.7134146 (42)	total: 1.07s	remaining: 9.52s
101:	learn: 0.7629179	test: 0.7073171	best: 0.7134146 (42)	total: 1.08s	remaining: 9.49s
102:	learn: 0.7629179	test: 0.7073171	best: 0.7134146 (42)	total: 1.09s	remaining: 9.47s
103:	learn: 0.7629179	test: 0.7073171	best: 0.7134146 (42)	total: 1.09s	remaining: 9.44s
104:	learn: 0.7659574	test: 0.7073171	best: 0.7134146 (42)	total: 1.1s	remaining: 9.42s
105:	learn: 0.7674772	test: 0.7073171	best: 0.7134146 (42)	total: 1.11s	remaining: 9.41s
106:	learn: 0.7659574	test:

198:	learn: 0.7978723	test: 0.7195122	best: 0.7195122 (119)	total: 2.02s	remaining: 8.12s
199:	learn: 0.7978723	test: 0.7195122	best: 0.7195122 (119)	total: 2.02s	remaining: 8.1s
200:	learn: 0.7993921	test: 0.7195122	best: 0.7195122 (119)	total: 2.04s	remaining: 8.09s
201:	learn: 0.7978723	test: 0.7195122	best: 0.7195122 (119)	total: 2.04s	remaining: 8.07s
202:	learn: 0.7993921	test: 0.7195122	best: 0.7195122 (119)	total: 2.05s	remaining: 8.06s
203:	learn: 0.8009119	test: 0.7195122	best: 0.7195122 (119)	total: 2.06s	remaining: 8.05s
204:	learn: 0.7993921	test: 0.7195122	best: 0.7195122 (119)	total: 2.07s	remaining: 8.04s
205:	learn: 0.8009119	test: 0.7195122	best: 0.7195122 (119)	total: 2.08s	remaining: 8.03s
206:	learn: 0.8009119	test: 0.7195122	best: 0.7195122 (119)	total: 2.09s	remaining: 8.01s
207:	learn: 0.8009119	test: 0.7195122	best: 0.7195122 (119)	total: 2.1s	remaining: 8s
208:	learn: 0.8024316	test: 0.7195122	best: 0.7195122 (119)	total: 2.11s	remaining: 7.99s
209:	learn: 0.8

308:	learn: 0.8282675	test: 0.7256098	best: 0.7256098 (246)	total: 3.03s	remaining: 6.78s
309:	learn: 0.8297872	test: 0.7256098	best: 0.7256098 (246)	total: 3.04s	remaining: 6.77s
310:	learn: 0.8297872	test: 0.7256098	best: 0.7256098 (246)	total: 3.05s	remaining: 6.76s
311:	learn: 0.8297872	test: 0.7256098	best: 0.7256098 (246)	total: 3.06s	remaining: 6.75s
312:	learn: 0.8313070	test: 0.7256098	best: 0.7256098 (246)	total: 3.07s	remaining: 6.74s
313:	learn: 0.8313070	test: 0.7256098	best: 0.7256098 (246)	total: 3.08s	remaining: 6.73s
314:	learn: 0.8313070	test: 0.7256098	best: 0.7256098 (246)	total: 3.09s	remaining: 6.72s
315:	learn: 0.8313070	test: 0.7256098	best: 0.7256098 (246)	total: 3.1s	remaining: 6.71s
316:	learn: 0.8313070	test: 0.7256098	best: 0.7256098 (246)	total: 3.11s	remaining: 6.7s
317:	learn: 0.8313070	test: 0.7256098	best: 0.7256098 (246)	total: 3.12s	remaining: 6.7s
318:	learn: 0.8313070	test: 0.7256098	best: 0.7256098 (246)	total: 3.13s	remaining: 6.68s
319:	learn: 0

414:	learn: 0.8632219	test: 0.7256098	best: 0.7256098 (246)	total: 4.05s	remaining: 5.71s
415:	learn: 0.8632219	test: 0.7256098	best: 0.7256098 (246)	total: 4.06s	remaining: 5.7s
416:	learn: 0.8617021	test: 0.7256098	best: 0.7256098 (246)	total: 4.07s	remaining: 5.68s
417:	learn: 0.8632219	test: 0.7256098	best: 0.7256098 (246)	total: 4.08s	remaining: 5.67s
418:	learn: 0.8632219	test: 0.7256098	best: 0.7256098 (246)	total: 4.08s	remaining: 5.66s
419:	learn: 0.8647416	test: 0.7256098	best: 0.7256098 (246)	total: 4.09s	remaining: 5.65s
420:	learn: 0.8647416	test: 0.7256098	best: 0.7256098 (246)	total: 4.1s	remaining: 5.64s
421:	learn: 0.8647416	test: 0.7256098	best: 0.7256098 (246)	total: 4.11s	remaining: 5.63s
422:	learn: 0.8647416	test: 0.7256098	best: 0.7256098 (246)	total: 4.12s	remaining: 5.62s
423:	learn: 0.8647416	test: 0.7256098	best: 0.7256098 (246)	total: 4.13s	remaining: 5.61s
424:	learn: 0.8632219	test: 0.7256098	best: 0.7256098 (246)	total: 4.14s	remaining: 5.6s
425:	learn: 0

...

520:	learn: 0.8662614	test: 0.7804878	best: 0.7987805 (139)	total: 4.88s	remaining: 4.48s
521:	learn: 0.8677812	test: 0.7804878	best: 0.7987805 (139)	total: 4.88s	remaining: 4.47s
522:	learn: 0.8693009	test: 0.7804878	best: 0.7987805 (139)	total: 4.89s	remaining: 4.46s
523:	learn: 0.8662614	test: 0.7804878	best: 0.7987805 (139)	total: 4.9s	remaining: 4.45s
524:	learn: 0.8677812	test: 0.7804878	best: 0.7987805 (139)	total: 4.91s	remaining: 4.44s
525:	learn: 0.8677812	test: 0.7804878	best: 0.7987805 (139)	total: 4.92s	remaining: 4.43s
526:	learn: 0.8708207	test: 0.7804878	best: 0.7987805 (139)	total: 4.93s	remaining: 4.42s
527:	learn: 0.8708207	test: 0.7804878	best: 0.7987805 (139)	total: 4.94s	remaining: 4.42s
528:	learn: 0.8723404	test: 0.7804878	best: 0.7987805 (139)	total: 4.95s	remaining: 4.41s
529:	learn: 0.8723404	test: 0.7804878	best: 0.7987805 (139)	total: 4.96s	remaining: 4.39s
530:	learn: 0.8723404	test: 0.7804878	best: 0.7987805 (139)	total: 4.97s	remaining: 4.39s
531:	learn:

630:	learn: 0.8920973	test: 0.7804878	best: 0.7987805 (139)	total: 5.88s	remaining: 3.44s
631:	learn: 0.8920973	test: 0.7804878	best: 0.7987805 (139)	total: 5.88s	remaining: 3.42s
632:	learn: 0.8920973	test: 0.7804878	best: 0.7987805 (139)	total: 5.89s	remaining: 3.42s
633:	learn: 0.8936170	test: 0.7804878	best: 0.7987805 (139)	total: 5.9s	remaining: 3.41s
634:	learn: 0.8951368	test: 0.7804878	best: 0.7987805 (139)	total: 5.91s	remaining: 3.4s
635:	learn: 0.8951368	test: 0.7804878	best: 0.7987805 (139)	total: 5.92s	remaining: 3.39s
636:	learn: 0.8951368	test: 0.7804878	best: 0.7987805 (139)	total: 5.93s	remaining: 3.38s
637:	learn: 0.8951368	test: 0.7804878	best: 0.7987805 (139)	total: 5.94s	remaining: 3.37s
638:	learn: 0.8966565	test: 0.7804878	best: 0.7987805 (139)	total: 5.95s	remaining: 3.36s
639:	learn: 0.8951368	test: 0.7804878	best: 0.7987805 (139)	total: 5.96s	remaining: 3.35s
640:	learn: 0.8951368	test: 0.7804878	best: 0.7987805 (139)	total: 5.96s	remaining: 3.34s
641:	learn: 

743:	learn: 0.9148936	test: 0.7804878	best: 0.7987805 (139)	total: 6.89s	remaining: 2.37s
744:	learn: 0.9148936	test: 0.7804878	best: 0.7987805 (139)	total: 6.9s	remaining: 2.36s
745:	learn: 0.9164134	test: 0.7804878	best: 0.7987805 (139)	total: 6.91s	remaining: 2.35s
746:	learn: 0.9164134	test: 0.7804878	best: 0.7987805 (139)	total: 6.92s	remaining: 2.34s
747:	learn: 0.9179331	test: 0.7804878	best: 0.7987805 (139)	total: 6.92s	remaining: 2.33s
748:	learn: 0.9179331	test: 0.7804878	best: 0.7987805 (139)	total: 6.93s	remaining: 2.32s
749:	learn: 0.9179331	test: 0.7804878	best: 0.7987805 (139)	total: 6.94s	remaining: 2.31s
750:	learn: 0.9179331	test: 0.7865854	best: 0.7987805 (139)	total: 6.95s	remaining: 2.31s
751:	learn: 0.9179331	test: 0.7865854	best: 0.7987805 (139)	total: 6.96s	remaining: 2.3s
752:	learn: 0.9179331	test: 0.7865854	best: 0.7987805 (139)	total: 6.97s	remaining: 2.29s
753:	learn: 0.9179331	test: 0.7865854	best: 0.7987805 (139)	total: 6.98s	remaining: 2.28s
754:	learn: 

852:	learn: 0.9316109	test: 0.7865854	best: 0.7987805 (139)	total: 7.91s	remaining: 1.36s
853:	learn: 0.9316109	test: 0.7865854	best: 0.7987805 (139)	total: 7.92s	remaining: 1.35s
854:	learn: 0.9331307	test: 0.7865854	best: 0.7987805 (139)	total: 7.92s	remaining: 1.34s
855:	learn: 0.9331307	test: 0.7865854	best: 0.7987805 (139)	total: 7.93s	remaining: 1.33s
856:	learn: 0.9331307	test: 0.7865854	best: 0.7987805 (139)	total: 7.94s	remaining: 1.32s
857:	learn: 0.9331307	test: 0.7865854	best: 0.7987805 (139)	total: 7.95s	remaining: 1.32s
858:	learn: 0.9331307	test: 0.7865854	best: 0.7987805 (139)	total: 7.96s	remaining: 1.31s
859:	learn: 0.9331307	test: 0.7865854	best: 0.7987805 (139)	total: 7.97s	remaining: 1.3s
860:	learn: 0.9331307	test: 0.7865854	best: 0.7987805 (139)	total: 7.98s	remaining: 1.29s
861:	learn: 0.9331307	test: 0.7865854	best: 0.7987805 (139)	total: 7.99s	remaining: 1.28s
862:	learn: 0.9331307	test: 0.7865854	best: 0.7987805 (139)	total: 8s	remaining: 1.27s
863:	learn: 0.

962:	learn: 0.9452888	test: 0.7865854	best: 0.7987805 (139)	total: 8.92s	remaining: 343ms
963:	learn: 0.9452888	test: 0.7865854	best: 0.7987805 (139)	total: 8.93s	remaining: 333ms
964:	learn: 0.9452888	test: 0.7865854	best: 0.7987805 (139)	total: 8.94s	remaining: 324ms
965:	learn: 0.9452888	test: 0.7865854	best: 0.7987805 (139)	total: 8.95s	remaining: 315ms
966:	learn: 0.9452888	test: 0.7865854	best: 0.7987805 (139)	total: 8.95s	remaining: 306ms
967:	learn: 0.9452888	test: 0.7865854	best: 0.7987805 (139)	total: 8.96s	remaining: 296ms
968:	learn: 0.9452888	test: 0.7865854	best: 0.7987805 (139)	total: 8.97s	remaining: 287ms
969:	learn: 0.9452888	test: 0.7865854	best: 0.7987805 (139)	total: 8.98s	remaining: 278ms
970:	learn: 0.9452888	test: 0.7865854	best: 0.7987805 (139)	total: 8.99s	remaining: 268ms
971:	learn: 0.9452888	test: 0.7865854	best: 0.7987805 (139)	total: 9s	remaining: 259ms
972:	learn: 0.9452888	test: 0.7865854	best: 0.7987805 (139)	total: 9.01s	remaining: 250ms
973:	learn: 0

In [218]:
model.predict([['this is a title','and the abstract']])

array([[0]])

In [219]:
filt_dat.label.value_counts()

0    243
8    166
7    146
6    102
1     67
5     44
3     29
4     22
2      3
Name: label, dtype: int64

## combining models

In [29]:
from catboost import CatBoostClassifier, Pool

In [33]:

print(mapp)

{0: 0, 1: 1, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9}


In [86]:
class BetterTogether():
    def __init__(self,rct_classifier,general_classifier):
        self.rct_model = rct_classifier
        self.gen_model = general_classifier
        
        mapp = {0:0,1:1}
        for i in range(2,9):
            mapp[i] = i+1
        self.gen_map = mapp
    def process(self,title,abstract):
        #step 1 
        rct_pred = self.rct_model.predict({"title": title, 
                        "abstract": abstract, 
                        "use_ptyp": False}, filter_type="balanced", filter_class="svm_cnn")
        
        #step 2
        if rct_pred[0]['is_rct'] == False:
            gen_pred = self.gen_model.predict([[title,abstract]])[0]
            
            output = np.argmax(gen_pred)
            return self.gen_map[output],0.9
        else:
            return 2,rct_pred[0]['score']
        

In [49]:
gen_model =CatBoostClassifier().load_model('cat1.cbm')

In [87]:
duo = BetterTogether(rct_clf,gen_model)

,cord_uid,title,abstract
0,zjufx4fo,Sequence requirements for RNA strand transfer ...,Nidovirus subgenomic mRNAs contain a leader se...
1,ymceytj3,"Crystal structure of murine sCEACAM1a[1,4]: a ...",CEACAM1 is a member of the carcinoembryonic an...
2,wzj2glte,Synthesis of a novel hepatitis C virus protein...,Hepatitis C virus (HCV) is an important human ...
3,2sfqsfm1,Structure of coronavirus main proteinase revea...,The key enzyme in coronavirus polyprotein proc...
4,i0zym7iq,Discontinuous and non-discontinuous subgenomic...,"Arteri-, corona-, toro- and roniviruses are ev..."
...,...,...,...
57361,rn4gm3sc,Structure-Based Stabilization of Non-native Pr...,[Image: see text] Structure-based stabilizatio...
57362,l5oecoot,Immunomodulators as an antimicrobial tool,The spectrum of infectious diseases has shifte...
57363,i4acg2ex,Discourses of disease: Representations of tube...,Abstract This paper critically examines the wa...
57364,rve0w3dg,Equine Infectious Anemia,This article describes clinical signs and tran...


In [62]:
testing = kag_df.merge(metadf[['cord_uid','title','abstract']],on='cord_uid').dropna()

In [88]:
res = {
    'y_true': [],
    'y_pred': [],
    'y_prob': [],
}
for i,row in testing.iterrows():
    y_pred,y_prob = duo.process(row['title'],row['abstract'])
    res['y_true'].append(row['label'])
    res['y_pred'].append(y_pred)
    res['y_prob'].append(y_prob)

In [67]:
results = pd.DataFrame(res)

In [69]:
results.head()

,y_true,y_pred,y_prob
0,0,0,0.629426
1,0,0,0.699518
2,0,0,0.687530
3,0,0,0.592347
4,0,0,0.645081


In [70]:
from sklearn import metrics

In [80]:
results.y_true.value_counts()

0    243
9    166
8    146
7    102
1     67
2     55
6     44
4     29
5     22
3      3
Name: y_true, dtype: int64

In [82]:
report = metrics.classification_report(results.y_true.values,
                              results.y_pred.values
                            )

/home/axhue/miniconda2/envs/rbtsearch/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [84]:
import pprint

pp = pprint.pprint

In [85]:
pp(report)

('             precision    recall  f1-score   support\n'
 '\n'
 '          0       0.37      0.95      0.54       243\n'
 '          1       0.57      0.06      0.11        67\n'
 '          2       0.95      0.98      0.96        55\n'
 '          3       0.00      0.00      0.00         3\n'
 '          4       0.83      0.17      0.29        29\n'
 '          5       0.00      0.00      0.00        22\n'
 '          6       1.00      0.14      0.24        44\n'
 '          7       0.91      0.41      0.57       102\n'
 '          8       0.56      0.27      0.37       146\n'
 '          9       0.92      0.34      0.50       166\n'
 '\n'
 'avg / total       0.66      0.50      0.46       877\n')
